In [1]:
!pip install -q langchain_google_genai

In [2]:
!pip install -q langchain

In [3]:
!pip install -q langchain_community

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [9]:
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_API_KEY"]=LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"]="pr-wilted-endoderm-31"
os.environ["GROQ_API_KEY"]=GROQ_API_KEY

In [10]:
!pip install -q langchain_groq

In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 55, 'total_tokens': 64, 'completion_time': 0.032727273, 'prompt_time': 0.00871019, 'queue_time': 0.0271147, 'total_time': 0.041437463}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4196e754db', 'finish_reason': 'stop', 'logprobs': None}, id='run-85e44eab-b29e-4dd5-97d1-23d08d6392e8-0', usage_metadata={'input_tokens': 55, 'output_tokens': 9, 'total_tokens': 64})

In [20]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)
chain = prompt | llm

chain.invoke(
    {
        "input_language": "English",
        "output_language": "Hindi",
        "input": "I love programming.",
    }
)

AIMessage(content='मुझे प्रोग्रामिंग बहुत पसंद है। (Mujhe programming bahut pasand hai.)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 50, 'total_tokens': 78, 'completion_time': 0.133495598, 'prompt_time': 0.013387055, 'queue_time': 0.018582854999999995, 'total_time': 0.146882653}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3884478861', 'finish_reason': 'stop', 'logprobs': None}, id='run-deb77a80-52aa-49a5-a178-52072054905e-0', usage_metadata={'input_tokens': 50, 'output_tokens': 28, 'total_tokens': 78})

In [26]:
!pip install -q langgraph 

In [34]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from langchain_groq import ChatGroq

# Define the tools for the agent to use
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


tools = [search]
model = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

app = create_react_agent(model, tools, checkpointer=checkpointer)

# Use the agent
final_state = app.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

"I'm just an AI, I don't have have access to real-time information. The search results would provide the most up-to-date and accurate information about the current weather in San Francisco."

In [30]:
final_state = app.invoke(
    {"messages": [{"role": "user", "content": "what about ny"}]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

"I'm just an AI, I don't have have access to real-time information, but I can tell you that New York City has a humid subtropical climate, with hot summers and cold winters. If you're looking for the most up-to-date weather information, I recommend checking a weather website or app."